In [ ]:
!pip install langchain openai wikipedia langchain-community "httpx==0.27.2"

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = "sk" # sk-로 시작하는 API 키를 입력하세요

llm = ChatOpenAI(model="gpt-4o-2024-11-20")

In [ ]:
#제로샷 ReAct 에이전트
from langchain.agents import initialize_agent, load_tools, AgentType
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True) # 대화형 에이전트를 초기화
agent.invoke("소나무 옮겨심기 좋은 계절은?") # 에이전트에게 질문을 전달하고 답변을 생성하도록 실행

In [ ]:
#Conversational ReAct
from langchain.memory import ConversationBufferMemory
tools = load_tools(["llm-math"],  llm=llm) # 에이전트가 사용할 도구 지정, 수학 관련 문제를 해결하는 도구
memory = ConversationBufferMemory(memory_key="chat_history") # 대화 기록을 저장할 키를 지정
conversational_agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True, # 디버깅 목적으로 에이전트의 내부 동작을 출력
    max_iterations=3, # 에이전트가 질문을 해결하기 위해 도구를 사용할 수 있는 최대 반복 횟수를 3으로 지
    memory=memory,)
conversational_agent.invoke("에드시런은 누구이며 2025년 현재 나이는 몇살이야?")

In [ ]:
#Self Ask With Search Agent 사용
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = "tvly" # tvly로 시작하는 API 키를 입력하세요

In [ ]:
tools = [TavilyAnswer(max_results=5, name="Intermediate Answer")] # 도구 이름을 설정하여 나중에 에이전트가 이 도구를 참조할 때 사용
prompt = hub.pull("hwchase17/self-ask-with-search") # 랭체인에서 제공하는 프롬프트 허브에서 self-ask-with-search 프롬프트를 가져옵니

agent = create_self_ask_with_search_agent(llm, tools, prompt) # "Self Ask With Search" 기능을 구현하는 에이전트를 생성

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True) #AgentExecutor 이용하여 에이전트 사용
agent_executor.invoke({"input": "AI Agent를 세 문장으로 설명해줘"})

In [ ]:
# ReAct document store
from langchain.docstore.wikipedia import Wikipedia # 문서 저장소로 위키피디아 이용
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer # 문서가 저장된 데이터베이스(위키피디아)를 탐색하는 도구

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,  # 용어에 대한 키워드 기반의 검색, 사용자가 정확히 무엇을 찾고자 하는지 모를 때 유용
    description="docstore에서 용어를 검색하세요.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup, # 고유하게 식별할 수 있는 키를 이용한 검색, 특정 문서(또는 항목)를 정확하게 찾을 때 사용
    description="docstore에서 용어 검색.",
  )
] # 에이전트가 사용할 수 있는 도구를 정의

In [ ]:
react = initialize_agent(tools = tools,
                         llm = llm,
                         agent = AgentType.REACT_DOCSTORE, # 에이전트 유형을 ReAct document store로 지정
                         handle_parsing_errors=True, # 에이전트가 작업을 수행하는 동안 발생하는 구문 오류(Parsing Errors)를 처리
                         max_iterations=1, # 에이전트가 최대 몇 번까지 작업을 반복할지 지정
                         max_execution_time=1, # 에이전트가 작업을 수행하는 최대 시간을 초 단위로 설정
                         verbose = True,) # 랭체인 에이전트 초기화

In [ ]:
def query_data(query):
    try:
        response = react.invoke(query)  # 에이전트에게 쿼리 전달
        print("----------------------", response)  # 결과 출력(디버깅용)
        return response  # 검색 결과 반환
    except Exception as e:
        print(f"Error: {e}")  # 에러 메시지를 출력
        raise  # 에러 다시 발생(추가 디버깅 용도)

query = "밍크 선인장 키우는 방법은?"
response = query_data(query)
print(response['output']) # 에이전트의 최종 답변 출력